## 0. House Keeping Functions

In [18]:
import csv
from scipy import ndimage

samples = []

# Assuming that DataSet are already downloaded
def prepare_samples(datasetpath):
    with open(datasetpath + '/driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        # Skip first row
        next(reader, None)
        for line in reader:
            # Update the image paths
            image_path = datasetpath + '/IMG/' + line[0].split('/')[-1]
            line[0] = image_path
            image_path = datasetpath + '/IMG/' + line[1].split('/')[-1]
            line[1] = image_path
            image_path = datasetpath + '/IMG/' + line[2].split('/')[-1]
            line[2] = image_path
            samples.append(line)


## 1. Prepare Samples 

In [19]:
import numpy as np

# Reset the list
samples.clear()

# Extract training data
prepare_samples('Data/DataSet1')
prepare_samples('Data/DataSet2')
prepare_samples('Data/DataSet3')
prepare_samples('Data/DataSet4')

## 2. Split Samples into '_Training Set_' & '_Validation Set_'

In [20]:
from sklearn.model_selection import train_test_split

# 80% training set and 20% validation set
Sample_train, Sample_valid = train_test_split(samples, test_size = 0.2)


## 3. Define Generator

In [21]:
import matplotlib.pyplot as plt
import sklearn    

def generator(samples, batch_size=32):
    num_samples = len(samples)
    
    while 1:  # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            images = []
            angles = []
            
            for batch_sample in batch_samples:
                
                steer_angle = float(batch_sample[3])
                ########## Center Image ##########
                image = plt.imread(batch_sample[0])
                images.append(image)
                angles.append(steer_angle)

                # Flip the image
                images.append(np.flipud(image))
                angles.append(float( steer_angle * (-1.0) ))
                
                
                ########## left Image ##########
                image = plt.imread(batch_sample[1])
                images.append(image)
                angles.append(steer_angle + 0.2)

                # Flip the image
                images.append(np.flipud(image))
                angles.append(float( (steer_angle + 0.2) * (-1) ))
                
                
                ########## Right Image ##########
                image = plt.imread(batch_sample[2])
                images.append(image)
                angles.append(steer_angle - 0.2)
                            
                # Flip the image
                images.append(np.flipud(image))
                angles.append(float( (steer_angle - 0.2) * (-1) ))
            
            X_train = np.array(images)
            y_train = np.array(angles)
            
            yield sklearn.utils.shuffle(X_train, y_train)
            
            
            
# compile and train the model using the generator function
train_generator = generator(Sample_train, batch_size=32)
validation_generator = generator(Sample_valid, batch_size=32)

## 4. Define A Model Using Keras

In [22]:
import keras
import tensorflow as tf

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 6} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [23]:
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers import Lambda, Cropping2D

model = Sequential()

# Normalize the data
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))

# trim image to only see section with road
model.add(Cropping2D( cropping=((70,25),(0,0)) ) )

# Convolutional Layer 1
model.add(Conv2D(filters=24, kernel_size=5, strides=(2, 2), activation='relu'))

# Convolutional Layer 2
model.add(Conv2D(filters=36, kernel_size=5, strides=(2, 2), activation='relu'))

# Convolutional Layer 3
model.add(Conv2D(filters=48, kernel_size=5, strides=(2, 2), activation='relu'))

# Convolutional Layer 4
model.add(Conv2D(filters=64, kernel_size=3, strides=(1, 1), activation='relu'))

# Convolutional Layer 5
model.add(Conv2D(filters=64, kernel_size=3, strides=(1, 1), activation='relu'))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(50))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('relu'))

model.add(Dense(1))

# Specify loss function and the optimizer for training
# we have used 'adam' optimizer with 'mean squared error' loss function
model.compile(loss='mse', optimizer='adam')


## 4. Train the model

In [24]:
import math

batch_size = 32

model.fit_generator(train_generator, steps_per_epoch=math.ceil(len(Sample_train)/batch_size), validation_data=validation_generator, validation_steps=math.ceil(len(Sample_valid)/batch_size), epochs=5, verbose=1)

# Save the model
model.save('model.h5')

print('Model Saved!')

# Print the model summary
model.summary()

Epoch 1/5
329/329 [==============================] - 64s - loss: 0.0220 - val_loss: 0.0158
Epoch 2/5
329/329 [==============================] - 64s - loss: 0.0153 - val_loss: 0.0136
Epoch 3/5
329/329 [==============================] - 63s - loss: 0.0127 - val_loss: 0.0121
Epoch 4/5
329/329 [==============================] - 63s - loss: 0.0112 - val_loss: 0.0111
Epoch 5/5
329/329 [==============================] - 63s - loss: 0.0102 - val_loss: 0.0109
Model Saved!
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_3 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_3 (Cropping2D)    (None, 65, 320, 3)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 31, 158, 24)       1824      
_________________________________________________________________
conv